In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def say_hello():
    response = client.chat.completions.create(
        model="gpt-5-nano",  # or another available model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Say hello to me in one short sentence."}
        ]
    )
    # Get the text of the assistant's reply
    return response.choices[0].message.content

if __name__ == "__main__":
    hello_message = say_hello()
    print(hello_message)


Hello there!


In [6]:
from newspaper import Article

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
]

def fetch_and_print(urls):
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()

            print(f"----- Text content from: {url} -----\n")
            print(article.text[:2000])
            print("\n\n")

        except Exception as e:
            print(f"Error fetching {url}: {e}")

if __name__ == "__main__":
    fetch_and_print(urls)

----- Text content from: https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity -----

For an empire that collapsed more than 1,500 years ago, ancient Rome maintains a powerful presence. About 1 billion people speak languages derived from Latin; Roman law shapes modern norms; and Roman architecture has been widely imitated. Christianity, which the empire embraced in its sunset years, remains the world’s largest religion. Yet all these enduring influences pale against Rome’s most important legacy: its fall. Had its empire not unravelled, or had it been replaced by a similarly overpowering successor, the world wouldn’t have become modern.

This isn’t the way that we ordinarily think about an event that has been lamented pretty much ever since it happened. In the late 18th century, in his monumental work The History of the Decline and Fall of the Roman Empire (1776-1788), the British historian Edward Gibbon called it ‘the greatest, perhaps, and most awful scen

In [10]:
import os
from newspaper import Article
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
]

def fetch_articles(urls):
    articles = []
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()

            articles.append({
                "url": url,
                "text": article.text
            })

        except Exception as e:
            print(f"Error fetching {url}: {e}")

    return articles


def generate_podcast_script(articles):
    sources_text = ""
    for i, a in enumerate(articles, start=1):
        sources_text += f"\nSOURCE {i} ({a['url']}):\n{a['text']}\n"

    prompt = f"""
You are a podcast scriptwriter.

Task:
Using the sources below, write an ORIGINAL podcast script suitable for a
5–10 minute episode.

Guidelines:
- Target audience: curious general listeners
- Tone: clear, engaging, slightly narrative
- Do NOT copy sentences or paragraphs from the sources
- Rephrase ideas in your own words
- Structure:
  1. Hook (30–45 seconds)
  2. Context and background
  3. 3–5 key ideas with explanations
  4. Short recap
  5. Reflective closing question

Optional:
- Light host cues like [PAUSE], [MUSIC IN], [MUSIC OUT]

Sources:
{sources_text}
"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You write engaging podcast scripts."},
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content


if __name__ == "__main__":
    articles = fetch_articles(urls)
    script = generate_podcast_script(articles)

    print("\n===== GENERATED PODCAST SCRIPT =====\n")
    print(script)

    speech = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=script[:4000]
)

    output_file = "podcast_episode.mp3"

    with open(output_file, "wb") as f:
        f.write(speech.read())

    print(f"Audio file created: {output_file}")



===== GENERATED PODCAST SCRIPT =====

[MUSIC IN — a light, reflective tune fades in]

Host: Imagine standing at the edge of a vast, once-mighty empire. The roads you’ve known fade into ruins, the coin you once used barely clinks in your pocket, and new rulers arrive with unfamiliar banners. It sounds like a catastrophe. But what if that very moment—the collapse—wasn’t the end of something, but the opening of something else entirely? A doorway through which a very different kind of world could begin to take shape. Welcome to Across the Ruins, where we ask how the fall of an empire helped spark a path to our modern world.

[PAUSE, MUSIC FADES]

Context and background

Host: We’re talking about the fall of Rome in the western half of its empire—a transition that didn’t just rewrite maps, but rewired how power, money, and ideas circulated. By the late 5th century, central Rome’s grip loosened. Germanic kingdoms emerged on old soil, and the “Roman state” as a single, centralized machine ga


===== GENERATED PODCAST SCRIPT (preview) =====

[INTRO MUSIC IN]

Host: Imagine for a moment that a single collapse, not a single invention, quietly redirected the course of history. A collapse so vast that it unstitched an empire, rewove power across dozens of kingdoms, and, over centuries, paved the roads we still walk today. Welcome to our podcast: the crossroads where ancient Rome’s fall meets the making of modern life. Today, we’re asking: could the fall of Rome have been less a tragedy and more a crucial opening act for the world that followed?

[PAUSE]

Context and background

Host: For more than a thousand years, Rome stood as the gravitational center of a vast world. In the West, its administrative machine, its roads, its tax system, even its language and law, left a lasting imprint. Then, around the late 5th century, that machine cracked. A long cascade of migrations, local power grabs, and shifting loyalties transformed the empire’s core into a patchwork of kingdoms—Goths, 

RuntimeError: ffmpeg concat failed.
STDOUT:


STDERR:
ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 11.1.0
  configuration: --prefix=/Users/elsa/miniconda3/envs/py311env --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1627813741069/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100


In [13]:
import os
import re
import shutil
import subprocess
from pathlib import Path

from dotenv import load_dotenv
from newspaper import Article
from openai import OpenAI

# -----------------------------
# Config
# -----------------------------
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
]

OUTPUT_DIR = Path("podcast_output")
CHUNKS_DIR = OUTPUT_DIR / "chunks"
FINAL_AUDIO = OUTPUT_DIR / "podcast_episode_final.mp3"

TTS_MODEL = "tts-1"
TTS_VOICE = "alloy"

# Keep headroom for TTS limits
MAX_CHARS_PER_CHUNK = 3500


# -----------------------------
# Fetch + Script generation
# -----------------------------
def fetch_articles(urls):
    articles = []
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()
            articles.append({"url": url, "text": article.text})
        except Exception as e:
            print(f"Error fetching {url}: {e}")
    return articles


def generate_podcast_script(articles):
    sources_text = ""
    for i, a in enumerate(articles, start=1):
        sources_text += f"\nSOURCE {i} ({a['url']}):\n{a['text']}\n"

    prompt = f"""
You are a podcast scriptwriter.

Task:
Using the sources below, write an ORIGINAL podcast script suitable for a
5–10 minute episode.

Guidelines:
- Target audience: curious general listeners
- Tone: clear, engaging, slightly narrative
- Do NOT copy sentences or paragraphs from the sources
- Rephrase ideas in your own words
- Structure:
  1. Hook (30–45 seconds)
  2. Context and background
  3. 3–5 key ideas with explanations
  4. Short recap
  5. Reflective closing question

Optional:
- Light host cues like [PAUSE], [MUSIC IN], [MUSIC OUT]

Sources:
{sources_text}
"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You write engaging podcast scripts."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


# -----------------------------
# Chunking
# -----------------------------
def normalize_whitespace(text: str) -> str:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def chunk_text(text: str, max_chars: int = MAX_CHARS_PER_CHUNK) -> list[str]:
    """
    Chunk by paragraphs, then by sentences if needed.
    Avoids cutting mid-sentence when possible.
    Hard-slices only as a last resort.
    """
    text = normalize_whitespace(text)
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]

    chunks = []
    current = ""

    def flush():
        nonlocal current
        if current.strip():
            chunks.append(current.strip())
        current = ""

    sentence_split = re.compile(r"(?<=[.!?])\s+")

    for p in paragraphs:
        parts = [p]
        if len(p) > max_chars:
            parts = sentence_split.split(p)

        for part in parts:
            part = part.strip()
            if not part:
                continue

            candidate = part if not current else current + "\n\n" + part

            if len(candidate) <= max_chars:
                current = candidate
            else:
                flush()

                if len(part) > max_chars:
                    for i in range(0, len(part), max_chars):
                        piece = part[i : i + max_chars].strip()
                        if piece:
                            chunks.append(piece)
                else:
                    current = part

    flush()
    return chunks


# -----------------------------
# TTS
# -----------------------------
def tts_to_mp3(text: str, out_path: Path):
    speech = client.audio.speech.create(
        model=TTS_MODEL,
        voice=TTS_VOICE,
        input=text,
    )
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as f:
        f.write(speech.read())


# -----------------------------
# Audio concat (absolute paths)
# -----------------------------
def ffmpeg_available() -> bool:
    return shutil.which("ffmpeg") is not None


def _ffmpeg_concat_escape(path: Path) -> str:
    # concat demuxer lines are: file '...'
    # use absolute posix path + escape single quotes
    p = path.resolve().as_posix()
    return p.replace("'", r"'\''")


def concat_mp3_ffmpeg(mp3_files: list[Path], out_file: Path):
    """
    Robust concat:
      1) Try stream-copy concat (fast, may fail for MP3 chunks).
      2) Retry with re-encode (reliable).
    Uses absolute paths in concat_list to avoid path duplication issues.
    """
    out_file.parent.mkdir(parents=True, exist_ok=True)
    list_file = out_file.parent / "concat_list.txt"

    # Ensure all files exist before calling ffmpeg
    missing = [p for p in mp3_files if not p.exists()]
    if missing:
        raise FileNotFoundError(f"Missing chunk files: {missing}")

    with open(list_file, "w", encoding="utf-8") as f:
        for p in mp3_files:
            f.write(f"file '{_ffmpeg_concat_escape(p)}'\n")

    def run(cmd: list[str]):
        res = subprocess.run(cmd, capture_output=True, text=True)
        if res.returncode != 0:
            raise RuntimeError(
                "ffmpeg concat failed.\n"
                f"CMD: {' '.join(cmd)}\n\n"
                f"STDOUT:\n{res.stdout}\n\nSTDERR:\n{res.stderr}"
            )
        return res

    # 1) Stream-copy
    try:
        run([
            "ffmpeg", "-y",
            "-f", "concat", "-safe", "0",
            "-i", str(list_file),
            "-c", "copy",
            str(out_file),
        ])
        return
    except RuntimeError as e:
        print(str(e))
        print("\nRetrying with re-encode (more reliable)...\n")

    # 2) Re-encode
    run([
        "ffmpeg", "-y",
        "-f", "concat", "-safe", "0",
        "-i", str(list_file),
        "-vn",
        "-c:a", "libmp3lame",
        "-q:a", "2",
        "-ar", "44100",
        "-ac", "2",
        str(out_file),
    ])


def concat_mp3_pydub(mp3_files: list[Path], out_file: Path):
    from pydub import AudioSegment  # pip install pydub

    combined = AudioSegment.empty()
    for p in mp3_files:
        combined += AudioSegment.from_file(p, format="mp3")

    out_file.parent.mkdir(parents=True, exist_ok=True)
    combined.export(out_file, format="mp3")


# -----------------------------
# Main
# -----------------------------
def main():
    OUTPUT_DIR.mkdir(exist_ok=True)
    CHUNKS_DIR.mkdir(parents=True, exist_ok=True)

    articles = fetch_articles(urls)
    if not articles:
        raise RuntimeError("No articles fetched. Check URLs / connectivity.")

    script = generate_podcast_script(articles)

    print("\n===== GENERATED PODCAST SCRIPT (preview) =====\n")
    print(script[:2000] + ("\n...\n" if len(script) > 2000 else ""))

    chunks = chunk_text(script, max_chars=MAX_CHARS_PER_CHUNK)
    print(f"\nChunked script into {len(chunks)} parts (<= {MAX_CHARS_PER_CHUNK} chars each).")

    mp3_files = []
    for idx, chunk in enumerate(chunks, start=1):
        mp3_path = CHUNKS_DIR / f"chunk_{idx:02d}.mp3"
        print(f"Generating audio for chunk {idx}/{len(chunks)} -> {mp3_path}")
        tts_to_mp3(chunk, mp3_path)
        mp3_files.append(mp3_path)

    print("\nCombining chunks into final audio...")

    if ffmpeg_available():
        concat_mp3_ffmpeg(mp3_files, FINAL_AUDIO)
    else:
        print("ffmpeg not found in PATH. Trying pydub fallback...")
        concat_mp3_pydub(mp3_files, FINAL_AUDIO)

    print(f"\nDone. Final audio file created: {FINAL_AUDIO.as_posix()}")


if __name__ == "__main__":
    main()



===== GENERATED PODCAST SCRIPT (preview) =====

[INTRO MUSIC IN]

Host: Welcome to The What If of History. I’m your guide, and today we’re asking a big, sometimes uncomfortable question: what if the fall of Rome wasn’t the end of an empire so much as the opening act for modern life as we know it?

Hook (30–45 seconds)
Imagine a patchwork of kingdoms spreading across a continent where one giant imperial machine used to loom. No single ruler could lock everything down, no vast tax machine to fund a standing army. What if that fragmentation—not unity—was the very condition that let science, commerce, and representative politics take root? It sounds counterintuitive, but many historians argue this messy disassembly was the precondition for Europe to become a cradle of modernity. Today, we’ll pull back the curtain on how the fall of Rome didn’t erase civilization so much as it reshaped the stage on which civilization could improvise, experiment, and eventually flourish.

Context and backgr